# Highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data

This notebook is primarily an intermediate to introducing to the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers..ipynb). Here the steps to see the difference between complexes represented in hu.MAP 2.0 and hu.MAP 3.0 data is done with one example. You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins. In ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb), it will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data. And that is more likely where you want to invest your time. You may want to quickly san the notebook below though to see the types of reports you'll expect when you give the snakemake file your list of identifiers. And with more details here it may help you troubleshoot such snakemake-generated reports. 

This effort just is meant to highlight possible differences. There is some approximating done to determine possible corresponding complexes that inherently brings in some possible judgement calls. You'll need to explore the results from each for yourself to compare in depth. The hu.MAP 2.0 data can be explored in sessions launched from my [humap2-binder repo](https://github.com/fomightez/humap2-binder).

------------

## Step #1: Preparation

The preparation parallels the notebooks ['Use of hu.MAP 3.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb) and ['Use of hu.MAP 2.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb), and so see them for more insight.

Get the data for hu.MAP 2.0 and hu.MAP 3.0:

In [1]:
!curl -OL https://raw.githubusercontent.com/fomightez/humap2-binder/refs/heads/main/additional_nbs/standardizing_initial_data/humap2_complexes_20200809InOrderMatched.csv
!curl -OL https://raw.githubusercontent.com/fomightez/humap3-binder/refs/heads/main/additional_nbs/standardizing_initial_data/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  502k  100  502k    0     0  1870k      0 --:--:-- --:--:-- --:--:-- 1875k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1243k  100 1243k    0     0  2426k      0 --:--:-- --:--:-- --:--:-- 2424k


Get an accessory script for adding information about the proteins in the complexes:

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/make_lookup_table_for_extra_info4complexes.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2893  100  2893    0     0  16392      0 --:--:-- --:--:-- --:--:-- 16344


##### Put the data on the complexes into Pandas dataframe

In [3]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3547      0 --:--:-- --:--:-- --:--:--  3558


In [3]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809InOrderMatched.csv
import pandas as pd
rd2_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
!uv run complexes_rawCSV_to_df.py hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv
rd3_df = pd.read_pickle('raw_complexes_pickled_df.pkl')

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               



--------

## Analyze complexes in hu.MAP 2.0 vs. hu.MAP 3.0 for a protein


In [4]:
search_term = "ROGDI"

Get complexes that protein is in in both sets of data, adding extra information like done in earlier notebooks.  
Then use that to make a summary table for change from version 2.0 to version 3.0.

In [5]:
# Next few cells will run the query collecting all complexes it occurs with and adding details for both datasets
pattern = fr'\b{search_term}\b' # Create a regex pattern with word boundaries
d2_rows_with_term_df = rd2_df[rd2_df['Uniprot_ACCs'].str.contains(pattern, case=False, regex=True) | rd2_df['genenames'].str.contains(pattern, case=False, regex=True)].copy()
d3_rows_with_term_df = rd3_df[rd3_df['Uniprot_ACCs'].str.contains(pattern, case=False, regex=True) | rd3_df['genenames'].str.contains(pattern, case=False, regex=True)].copy()
# make the dataframe have each row be a single protein
# to prepare to use pandas `explode()` to do that, first make the content in be lists
d2_rows_with_term_df['Uniprot_ACCs'] = d2_rows_with_term_df['Uniprot_ACCs'].str.split()
d3_rows_with_term_df['Uniprot_ACCs'] = d3_rows_with_term_df['Uniprot_ACCs'].str.split()
d2_rows_with_term_df['genenames'] = d2_rows_with_term_df['genenames'].str.split()
d3_rows_with_term_df['genenames'] = d3_rows_with_term_df['genenames'].str.split()
# Now use explode to create a new row for each element in both columns
df2_expanded = d2_rows_with_term_df.explode(['Uniprot_ACCs', 'genenames']).copy()
df3_expanded = d3_rows_with_term_df.explode(['Uniprot_ACCs', 'genenames']).copy()
# Reset the index 
df2_expanded = df2_expanded.reset_index(drop=True)
df3_expanded = df3_expanded.reset_index(drop=True)
# Display the first few rows of the expanded dataframe
print(df2_expanded.tail())
print(" ")
print(df3_expanded.tail())
# Next add extra information from UniProt for each protein

       HuMAP2_ID  Confidence Uniprot_ACCs genenames
6   HuMAP2_01834           3       Q9Y4E6      WDR7
7   HuMAP2_03388           2       Q9Y485     DMXL1
8   HuMAP2_03388           2       Q9GZN7     ROGDI
9   HuMAP2_03388           2       Q8TDJ6     DMXL2
10  HuMAP2_03388           2       Q9Y4E6      WDR7
 
          HuMAP3_ID  ComplexConfidence Uniprot_ACCs genenames
297  huMAP3_13872.1                  6       Q8NI08     NCOA7
298  huMAP3_13872.1                  6       Q8TDJ6     DMXL2
299  huMAP3_13872.1                  6       Q9GZN7     ROGDI
300  huMAP3_13872.1                  6       Q9Y485     DMXL1
301  huMAP3_13872.1                  6       Q9Y4E6      WDR7


In [6]:
# This cell makes lookup table with the extra information; it takes a while to run & so is in a cell on its own to save time during development
df_expanded = pd.concat([df2_expanded,df3_expanded])
%run -i make_lookup_table_for_extra_info4complexes.py

In [7]:
# Use collected information to enhance the dataframes
pn_dict = {k: v['protein_name'] for k, v in lookup_dict.items()}
disease_dict = {k: v['disease'] for k, v in lookup_dict.items()}
synonyms_dict = {k: v['synonyms'] for k, v in lookup_dict.items()}
df3_expanded['synonyms'] = df3_expanded['Uniprot_ACCs'].map(synonyms_dict)
df3_expanded['protein_name'] = df3_expanded['Uniprot_ACCs'].map(pn_dict)
df3_expanded['disease'] = df3_expanded['Uniprot_ACCs'].map(disease_dict)
conf_val2text_dict = {
    1: 'Extremely High',
    2: 'Very High',
    3: 'High',
    4: 'Moderate High',
    5: 'Medium High',
    6: 'Medium'
}
# Use vectorized mapping to convert confidence values to text
df3_expanded['ComplexConfidence'] = df3_expanded['ComplexConfidence'].map(conf_val2text_dict)
base_uniprot_url = 'https://www.uniprot.org/uniprotkb/'
df3_expanded = df3_expanded.assign(Link=base_uniprot_url + df3_expanded['Uniprot_ACCs'])

# do same for 2.0 data
df2_expanded['synonyms'] = df2_expanded['Uniprot_ACCs'].map(synonyms_dict)
df2_expanded['protein_name'] = df2_expanded['Uniprot_ACCs'].map(pn_dict)
df2_expanded['disease'] = df2_expanded['Uniprot_ACCs'].map(disease_dict)
conf_val2text_dict = {
    1: 'Extremely High',
    2: 'Very High',
    3: 'High',
    4: 'Moderate High',
    5: 'Medium High',
    6: 'Medium'
}
# Use vectorized mapping to convert confidence values to text
df2_expanded['Confidence'] = df2_expanded['Confidence'].map(conf_val2text_dict)
base_uniprot_url = 'https://www.uniprot.org/uniprotkb/'
df2_expanded = df2_expanded.assign(Link=base_uniprot_url + df2_expanded['Uniprot_ACCs'])

In [8]:
# Try combining, because ranked by size probably correct if the number of shared items doesn't get any better if you re-order. Then fall back to ranking by share members
# if doesn't seem to work that the shared items never gets any better from ranked size comparisons.
# Get the groups from both dataframes
df2_groups = df2_expanded.groupby('HuMAP2_ID')['Uniprot_ACCs'].apply(list)
df3_groups = df3_expanded.groupby('HuMAP3_ID')['Uniprot_ACCs'].apply(list)

# Convert lists to sets
source1_sets = {idx: set(ids) for idx, ids in df2_groups.items()}
source2_sets = {idx: set(ids) for idx, ids in df3_groups.items()}

# Sort groups by size
source1_sorted = sorted(source1_sets.items(), key=lambda x: len(x[1]), reverse=True)
source2_sorted = sorted(source2_sets.items(), key=lambda x: len(x[1]), reverse=True)

# Function to check shared items
def get_shared_items(set1, set2):
    return set1.intersection(set2)

# Function for weighted similarity
def weighted_similarity(set1, set2, weight_jaccard=0.7):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard = len(intersection) / len(union) if union else 0
    max_possible_intersection = max(len(set1), len(set2))
    normalized_intersection = len(intersection) / max_possible_intersection if max_possible_intersection != 0 else 0
    
    return weight_jaccard * jaccard + (1 - weight_jaccard) * normalized_intersection

# Correspondence tracking
correspondences = []
used_source2_indices = set()

# First pass: Try to match by ranking and shared items
for i, (humap2_id, set1) in enumerate(source1_sorted):
    # If we've reached the end of source2 sorted list, break
    if i >= len(source2_sorted):
        break
    
    humap3_id, set2 = source2_sorted[i]
    
    # Check shared items
    shared = get_shared_items(set1, set2)
    
    if len(shared) > 1:  # need greater than 1 and not zero because always going to be greater than zero because every complex has to have query identifier
        correspondences.append({
            'HuMAP2_ID': humap2_id,
            'HuMAP3_ID': humap3_id,
            'HuMAP2_size': len(set1),
            'HuMAP3_size': len(set2),
            'shared_items': len(shared),
            'shared_elements': shared,
            'weighted_similarity': weighted_similarity(set1, set2)
        })
        used_source2_indices.add(i)

# Second pass: Fallback method for unmatched groups
# Create a list of unused source2 indices
unused_source2_indices = [
    j for j in range(len(source2_sorted)) 
    if j not in used_source2_indices
]

# Fallback matching
for humap2_id, set1 in source1_sorted:
    # Skip if this HuMAP2 ID has already been matched
    if any(corr['HuMAP2_ID'] == humap2_id for corr in correspondences):
        continue
    
    # Find best match among unused source2 groups
    best_match = None
    best_similarity = -1
    best_unused_index = -1
    
    for j in unused_source2_indices:
        humap3_id, set2 = source2_sorted[j]
        similarity = weighted_similarity(set1, set2)
        
        if similarity > best_similarity:
            best_match = (humap3_id, set2)
            best_similarity = similarity
            best_unused_index = j
    
    # Add the best match if found
    if best_match:
        humap3_id, set2 = best_match
        shared = get_shared_items(set1, set2)
        
        correspondences.append({
            'HuMAP2_ID': humap2_id,
            'HuMAP3_ID': humap3_id,
            'HuMAP2_size': len(set1),
            'HuMAP3_size': len(set2),
            'shared_items': len(shared),
            'shared_elements': shared,
            'weighted_similarity': best_similarity
        })
        
        # Remove this index from unused indices
        unused_source2_indices.remove(best_unused_index)

# Convert to DataFrame for easy viewing
df_correspondences = pd.DataFrame(correspondences)

# Sort by HuMAP2 ID size
df_correspondences = df_correspondences.sort_values('HuMAP2_size', ascending=False)


# Make summary table  with those correspondences
# Create a dictionary to map HuMAP2_ID to its corresponding details
correspondences_dict = {
    corr['HuMAP2_ID']: {
        'HuMAP3_ID': corr['HuMAP3_ID'],
        'size_df3': corr['HuMAP3_size'],
        'shared_items': corr['shared_items'],
        'weighted_similarity': corr['weighted_similarity']
    } for corr in correspondences
}

# Modify the correspondence creation to use this new matching
# First, create the base dataframe as before
df2_sorted = df2_expanded.groupby('HuMAP2_ID').size().sort_values(ascending=False)
df3_sorted = df3_expanded.groupby('HuMAP3_ID').size().sort_values(ascending=False)

# Convert the series to dataframes
df2_ranks = df2_sorted.reset_index()
df3_ranks = df3_sorted.reset_index()

# Rename columns for clarity
df2_ranks.columns = ['HuMAP2_ID', 'size_df2']
df3_ranks.columns = ['HuMAP3_ID', 'size_df3']

# Add rank columns
df2_ranks['rank'] = range(1, len(df2_ranks) + 1)
df3_ranks['rank'] = range(1, len(df3_ranks) + 1)

# Merge the rankings side by side
correspondence = pd.merge(
    df2_ranks,
    df3_ranks,
    on='rank',
    how='outer'
)
# Add details from the correspondences
def get_correspondence_details(row):
    if pd.notna(row['HuMAP2_ID']) and row['HuMAP2_ID'] in correspondences_dict:
        details = correspondences_dict[row['HuMAP2_ID']]
        return pd.Series({
            #'HuMAP3_ID': details['HuMAP3_ID'],
            #'size_df3': details['size_df3'],
            'shared_items': details['shared_items'],
            'improvement_in_v3': int(details['size_df3'] - row['size_df2'])
        })
    return pd.Series({
        #'HuMAP3_ID': '',
        #'size_df3': 0,
        'shared_items': 'na',
        'improvement_in_v3': 'na'
    })

correspondence_details = correspondence.apply(get_correspondence_details, axis=1)
correspondence = pd.concat([correspondence, correspondence_details], axis=1)
# Calculate coverage ratio
def cov_ratio(row):
    if isinstance(row['shared_items'], (int, float)) and row['size_df2'] > 0 and row['size_df3'] > 0:
        return round(row['shared_items'] / min(row['size_df2'], row['size_df3']) * 100, 2) 
    else:
        return 'na'

correspondence['coverage_ratio'] = correspondence.apply(cov_ratio, axis=1)

# Prepare display
correspondence = correspondence.sort_values('rank')
correspondence['size_df2'] = correspondence['size_df2'].fillna(0).astype(int)
correspondence['size_df3'] = correspondence['size_df3'].fillna(0).astype(int)


# Create display dataframe
display_df = correspondence.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')

# Update print statement
print("\nCorrespondences (considering shared complex members) for version 2 vs. 3 with improvement, shared items, and coverage ratio:")
print(display_df[['HuMAP2_ID', 'size_df2', 'rank', 'HuMAP3_ID', 'size_df3', 'improvement_in_v3', 'shared_items', 'coverage_ratio']].to_string(index=False))


Correspondences (considering shared complex members) for version 2 vs. 3 with improvement, shared items, and coverage ratio:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3 shared_items coverage_ratio
HuMAP2_01834        5     1 huMAP3_12042.1        56                51            4           80.0
HuMAP2_03388        4     2 huMAP3_10511.1        35                31            3           75.0
HuMAP2_01148        2     3 huMAP3_13872.1        29                27            2          100.0
                          4 huMAP3_09477.1        26                na           na             na
                          5 huMAP3_09873.1        24                na           na             na
                          6 huMAP3_08678.1        22                na           na             na
                          7 huMAP3_09242.1        20                na           na             na
                          8 huMAP3_07329.1        20                na           n

That above should be a summary table quickly giving a sense of the improvement in 3.0 data. (Or lack thereof, if yo have substituted with your protein of interest?)

Despite general improvment, I noticed some information present in version 2.0 complexes went missing in complexes reported in version 3.0.
    
#### Check for disappearing proteins in complexes

In [9]:
# Make sure nothing present in HuMAP2 data disappeared in huMAP3 data
# Function to compare UniProt ACCs between HuMAP2 and HuMAP3
def check_disappearing_items(df_correspondences, df2_expanded, df3_expanded):
    # Tracking disappearing items
    disappearing_items = []
    
    for _, correspondence in df_correspondences.iterrows():
        humap2_id = correspondence['HuMAP2_ID']
        humap3_id = correspondence['HuMAP3_ID']
        
        # Get UniProt ACCs for this HuMAP2 group
        humap2_accs = set(df2_expanded[df2_expanded['HuMAP2_ID'] == humap2_id]['Uniprot_ACCs'])
        
        # Get UniProt ACCs for this HuMAP3 group
        humap3_accs = set(df3_expanded[df3_expanded['HuMAP3_ID'] == humap3_id]['Uniprot_ACCs'])
        
        # Find items in HuMAP2 that are not in HuMAP3
        lost_items = humap2_accs - humap3_accs
        
        if lost_items:
            disappearing_items.append({
                'HuMAP2_ID': humap2_id,
                'HuMAP3_ID': humap3_id,
                'lost_items': lost_items,
                'total_humap2_items': len(humap2_accs),
                'total_humap3_items': len(humap3_accs),
                'percent_lost': len(lost_items) / len(humap2_accs) * 100 if humap2_accs else 0
            })
    
    # Convert to DataFrame for easy analysis
    df_disappearing = pd.DataFrame(disappearing_items)
    
    # Display results
    global something_dropped_completely
    something_dropped_completely = False
    if not df_disappearing.empty:
        print("\nDisappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:")
        for _, row in df_disappearing.iterrows():
            print(f"\nHuMAP2 ID: {row['HuMAP2_ID']} -> HuMAP3 ID: {row['HuMAP3_ID']}")
            print(f"Total items in HuMAP2: {row['total_humap2_items']}")
            print(f"Total items in HuMAP3: {row['total_humap3_items']}")
            print(f"Percent of items lost: {row['percent_lost']:.2f}%")
            print("UniProt ACCs Lost From Corresponding Complex:")
            for item in row['lost_items']:
                dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
                if dropped_completely:
                    print(f"  - {item} (AND NOT IN ANY OTHER Hu.MAP3.0 {search_term}-complexes!!!)")
                    something_dropped_completely = True
                else:
                    print(f"  - {item} (But present in other, {search_term}-related Hu.MAP3.0 complexes)")
        
        # Optional: Save to CSV
        df_disappearing.to_csv('disappearing_items.csv', index=False)
        
        return df_disappearing, something_dropped_completely
    else:
        print("\nNo disappearing items found between HuMAP2 and HuMAP3.")
        return None, something_dropped_completely

# Call the function
disappearing_df, something_dropped_completely = check_disappearing_items(df_correspondences, df2_expanded, df3_expanded)


Disappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:

HuMAP2 ID: HuMAP2_01834 -> HuMAP3 ID: huMAP3_12042.1
Total items in HuMAP2: 5
Total items in HuMAP3: 56
Percent of items lost: 20.00%
UniProt ACCs Lost From Corresponding Complex:
  - P50993 (AND NOT IN ANY OTHER Hu.MAP3.0 ROGDI-complexes!!!)

HuMAP2 ID: HuMAP2_03388 -> HuMAP3 ID: huMAP3_10511.1
Total items in HuMAP2: 4
Total items in HuMAP3: 35
Percent of items lost: 25.00%
UniProt ACCs Lost From Corresponding Complex:
  - Q8TDJ6 (But present in other, ROGDI-related Hu.MAP3.0 complexes)


In [10]:
# TO DO: Also make sure nothing that is in the majority of complexes in hu.MAP 2.0 data disappears entirely from hu.MAP 3.0 data.
# That can only occur as the case if `something_dropped_completely`, that we've already check on, is True
majority_item_dropped_completely = False #set up by declaring false
if something_dropped_completely:
    # collect information on what identifiers occur in the majority of complexes in hu.MAP 2.0 data
    # the search term should always be present and can thus serve as a sanity check for this collection
    majority_identifiers = []
    def find_majority_uniprot_accs(df):
        # Group by HuMAP2_ID and get unique Uniprot_ACCs for each complex
        complex_accs = df.groupby('HuMAP2_ID')['Uniprot_ACCs'].unique()
        
        # Count total number of unique complexes
        total_complexes = len(complex_accs)
        
        # Flatten and count occurrences of each Uniprot ACC across complexes
        all_accs = df['Uniprot_ACCs'].tolist()
        acc_counts = pd.Series(all_accs).value_counts()
        
        # Find ACCs that appear in more than 50% of complexes
        majority_accs = acc_counts[acc_counts >= 0.51 * total_complexes].index.tolist()
        
        return majority_accs
    if find_majority_uniprot_accs(df2_expanded):
        majority_identifiers = find_majority_uniprot_accs(df2_expanded)
    # Now check none of those `majority_identifiers` match the disappearing ones dropped completely
    print("\nHu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:")
    for item in list(set.union(*disappearing_df['lost_items'])): # note `list(set.union(*disappearing_df['lost_items']))` gets the unique set members from the 'lost_items' column
        if item in majority_identifiers:
            majority_item_dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
            if majority_item_dropped_completely:
                print(f"  - {item} IS IN MAJORITY OF Hu.MAP2.0 COMPLEXES YET NOT IN ANY Hu.MAP3.0 {search_term}-complexes!!! (PERHAPS CONCERNING?)")
                majority_item_dropped_completely = True
    if majority_item_dropped_completely == False:
        print("NONE")


Hu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:
NONE


 You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins, and so you'll want to first check out the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb). That notebook will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data.
 
-----

Enjoy!

See my [humap3-binder repo](https://github.com/fomightez/humap3-binder) and [humap3-utilities](https://github.com/fomightez/structurework/humap3-utilities) for related information & resources for this notebook.



-----
